In [19]:
import os
import shutil
from PIL import Image, ExifTags #pip install Pillow pillow-heif
from datetime import datetime
from pathlib import Path

In [20]:
import subprocess

In [21]:
try:
    from pillow_heif import register_heif_opener
    register_heif_opener()  # Enables reading HEIC/HEIF (iPhone photos)
except ImportError:
    print("Install pillow-heif for HEIC support: pip install pillow-heif")

In [22]:
def get_exif_date(image_path):
    """Try multiple EXIF date tags for images (fallback if DateTimeOriginal is missing)."""
    try:
        image = Image.open(image_path)
        exif_data = image._getexif()
        if exif_data:
            date_tags = ["DateTimeOriginal", "DateTimeDigitized", "DateTime"]
            for tag, value in exif_data.items():
                tag_name = ExifTags.TAGS.get(tag)
                if tag_name in date_tags:
                    try:
                        return datetime.strptime(value, "%Y:%m:%d %H:%M:%S")
                    except Exception:
                        continue
    except Exception:
        pass
    return None

In [23]:
def get_video_date(video_path):
    """Extract 'Date Taken' from video metadata using ffprobe (if available)."""
    try:
        result = subprocess.run(
            ["ffprobe", "-v", "error", "-show_entries",
             "format_tags=creation_time", "-of",
             "default=noprint_wrappers=1:nokey=1", video_path],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        date_str = result.stdout.strip()
        if date_str:
            try:
                return datetime.fromisoformat(date_str.replace("Z", "+00:00"))
            except ValueError:
                pass
    except FileNotFoundError:
        print("Install ffmpeg/ffprobe for video metadata support.")
    return None

In [24]:
def get_file_system_date(file_path):
    """Fallback to file creation/modification date if no EXIF/metadata available."""
    try:
        ts = os.path.getmtime(file_path)  # modification time (works cross-platform)
        return datetime.fromtimestamp(ts)
    except Exception:
        return None

In [25]:
def get_date_taken(file_path):
    """Get date taken from EXIF, video metadata, or fallback to file system date."""
    ext = file_path.lower()
    date_taken = None

    if ext.endswith((".jpg", ".jpeg", ".png", ".heic", ".heif")):
        date_taken = get_exif_date(file_path)
    elif ext.endswith((".mp4", ".mov", ".avi", ".mkv")):
        date_taken = get_video_date(file_path)

    if not date_taken:
        date_taken = get_file_system_date(file_path)

    return date_taken

In [26]:
def get_unique_filename(folder, filename):
    """Generate a unique filename if duplicate exists."""
    base, ext = os.path.splitext(filename)
    counter = 1
    new_filename = filename
    while os.path.exists(os.path.join(folder, new_filename)):
        new_filename = f"{base}({counter}){ext}"
        counter += 1
    return new_filename

In [27]:
def format_filename_by_date(date_taken, ext):
    """Format filename based on date and time."""
    return f"{date_taken.strftime('%Y-%m-%d_%H-%M-%S')}{ext.lower()}"

In [28]:
def sort_media_by_date(source_folder, destination_folder, rename_by_date=False):
    """Sort images & videos into folders by year and month based on metadata."""
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    unsorted_folder = os.path.join(destination_folder, "Unsorted")
    os.makedirs(unsorted_folder, exist_ok=True)

    for root, _, files in os.walk(source_folder):
        for file in files:
            file_path = os.path.join(root, file)

            if not file.lower().endswith((".jpg", ".jpeg", ".png", ".heic", ".heif", ".mp4", ".mov", ".avi", ".mkv")):
                continue

            date_taken = get_date_taken(file_path)
            if date_taken:
                folder_name = f"{date_taken.year}-{date_taken.month:02d}"
                target_folder = os.path.join(destination_folder, folder_name)
                new_filename = (
                    format_filename_by_date(date_taken, Path(file).suffix)
                    if rename_by_date else file
                )
            else:
                target_folder = unsorted_folder
                new_filename = file

            os.makedirs(target_folder, exist_ok=True)
            unique_name = get_unique_filename(target_folder, new_filename)
            shutil.copy2(file_path, os.path.join(target_folder, unique_name))
            print(f"Copied: {file} → {target_folder}/{unique_name}")

In [41]:
source = '/Users/max/Pictures/Temp/iphone_photos_unsorted'

In [42]:
destination = '/Users/max/Pictures/Temp/iphone_photos_sorted'

In [43]:
rename_option = 'y'

In [44]:
sort_media_by_date(source, destination, rename_by_date=rename_option)

Copied: IMG_E2663.JPG → /Users/max/Pictures/Temp/iphone_photos_sorted/2022-09/2022-09-27_06-34-55.jpg
Copied: IMG_2603.JPG → /Users/max/Pictures/Temp/iphone_photos_sorted/2022-09/2022-09-18_12-02-17.jpg
Copied: IMG_2641.MOV → /Users/max/Pictures/Temp/iphone_photos_sorted/2022-09/2022-09-21_16-45-20(2).mov
Copied: IMG_2494.MOV → /Users/max/Pictures/Temp/iphone_photos_sorted/2022-09/2022-09-17_13-15-24(1).mov
Copied: IMG_3723.JPG → /Users/max/Pictures/Temp/iphone_photos_sorted/2022-12/2022-12-30_22-19-22.jpg
Copied: IMG_E2529.JPG → /Users/max/Pictures/Temp/iphone_photos_sorted/2022-09/2022-09-18_10-34-02.jpg
Copied: IMG_2575.JPG → /Users/max/Pictures/Temp/iphone_photos_sorted/2022-09/2022-09-18_10-38-17.jpg
Copied: IMG_2561.JPG → /Users/max/Pictures/Temp/iphone_photos_sorted/2022-09/2022-09-18_10-37-54.jpg
Copied: IMG_2549.JPG → /Users/max/Pictures/Temp/iphone_photos_sorted/2022-09/2022-09-18_10-37-37.jpg
Copied: IMG_2537.MOV → /Users/max/Pictures/Temp/iphone_photos_sorted/2022-09/2022-0

In [ ]:
# if __name__ == "__main__":
#     source = input("Enter the path to the folder with unsorted images & videos: ").strip('"')
#     destination = input("Enter the path to the folder where sorted files will be saved: ").strip('"')
#     rename_option = input("Rename files by date & time taken? (y/n): ").strip().lower() == "y"

#     sort_media_by_date(source, destination, rename_by_date=rename_option)
#     print("\n✅ Sorting complete!")